# Fluent Python
https://github.com/fluentpython/example-code

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 6 Design Patterns with First-Class Functions
### Case Study: Refactoring Strategy

- Context <br>
    Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. In the ecommerce example, the context is an Order , which is configured to apply a promotional discount according to one of several algorithms.

- Strategy <br>
    The interface common to the components that implement the different algorithms.
    In our example, this role is played by an abstract class called Promotion .

- Concrete Strategy <br>
    One of the concrete subclasses of Strategy. FidelityPromo , BulkPromo , and Large OrderPromo are the three concrete strategies implemented.


Exemple

- order : context 
- promotion : strategy
- fidelity_promo :concrete strategy


abc 클래스를 이용하는 이유 

첫번째, abc 클래스를 이용하게 되면, 해당 BaseClass 는 인스턴스화 될 수 없다. 
단지 파생클래스 구현을 위한 추상화 기능 제공 역할을 하게 될 뿐이다.

두번째, abc 클래스를 이용하게 될 경우 에러 발생 시점이 다르다.
메서드에 raise를 이용해 NotImplementedError 를 선언해 놓은 경우에는 런타임 상황에서 해당 메서드가 실제로 호출이 되는 시점에 에러를 발생시키게 되지만, abc 를 이용하는 경우에는 해당 모듈이 import 되는 순간부터 에러를 발생시키게 된다. 

즉, abc 클래스를 이용하는 경우 조금 더 strict 한 모듈 관리가 가능해 진다는 점이다.

In [2]:
from abc import ABC, abstractmethod

class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

class Promotion(ABC):  # the Strategy: an Abstract Base Class

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""
        
class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

1. refactoring : promotion(ABC) class 제거 
    

In [6]:
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

2. 함수 리스트 반복으로 최대 할인액 알아내기 

In [7]:
# BEGIN STRATEGY_BEST

promos = [fidelity_promo, bulk_item_promo, large_order_promo]  # <1>

def best_promo(order):  # <2>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)  # <3>

# END STRATEGY_BEST

3. globals() 글로벌 심볼 테이블을 딕셔너리 형태로 돌려줌

In [8]:
promos = [globals()[name] for name in globals()  # <1>
            if name.endswith('_promo')  # <2>
            and name != 'best_promo']   # <3>

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)  # <4>

In [12]:
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

4. promotions 모듈 내부 조사하는 방법

```python
import inspect

promos = [func for name, func in
                inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)
```

<img src="Strategy.png">